In [ ]:
# E-commerce Exploratory Data Analysis (EDA)

## Project Overview
"""
This notebook performs comprehensive exploratory data analysis on the e-commerce dataset to uncover patterns, trends, and business insights.

### Analysis Sections:
    "1. Data Overview & Summary Statistics\n",
    "2. Sales Performance Analysis\n",
    "3. Customer Behavior Analysis\n",
    "4. Product Performance Analysis\n",
    "5. Time Series Analysis\n",
    "6. Geographic Analysis\n",
    "7. Customer Segmentation\n",
    "8. Business Insights & Recommendations"
"""
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Setup
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("📊 E-commerce EDA Started...")
print("📂 Loading processed data...")

transactions = pd.read_csv('../data/processed/enriched_transactions.csv')
customers = pd.read_csv('../data/processed/customers_clean.csv')
products = pd.read_csv('../data/processed/products_clean.csv')
customer_summary = pd.read_csv('../data/processed/customer_summary.csv')
product_performance = pd.read_csv('../data/processed/product_performance.csv')
monthly_sales = pd.read_csv('../data/processed/monthly_sales.csv')
    
# Convert date columns to datetime
transactions['date'] = pd.to_datetime(transactions['date'])
customers['signup_date'] = pd.to_datetime(customers['signup_date'])

print("✅ Data loaded successfully!")
print(f"📈 Dataset shapes: Transactions {transactions.shape}, Customers {customers.shape}, Products {products.shape}")
## 1. Data Overview & Summary Statistics"
print("📋 DATA OVERVIEW")
print("=" * 50)

# Basic info
print("\n📊 Transactions Dataset Info:")
print(f"Shape: {transactions.shape}")
print(f"Date Range: {transactions['date'].min()} to {transactions['date'].max()}")
print(f"Total Revenue: ${transactions['revenue'].sum():,.2f}")
print(f"Total Profit: ${transactions['profit'].sum():,.2f}")
print("\n")
print("\n📊 Key Metrics:")
print(f"• Average Transaction Value: ${transactions['revenue'].mean():.2f}")
print(f"• Average Items per Transaction: {transactions['quantity'].mean():.1f}")
print(f"• Total Unique Customers: {transactions['customer_id'].nunique()}")
print(f"• Total Unique Products: {transactions['product_id'].nunique()}")
print(f"• Most Common Customer Tier: {transactions['customer_tier'].mode().iloc[0]}")
print(f"• Most Popular Category: {transactions['category'].mode().iloc[0]}")
# Summary statistics\n",
print("\n📈 Numerical Summary Statistics:")
display(transactions[['quantity', 'price', 'revenue', 'profit']].describe())

print("\n📊 Categorical Variables Summary:")
categorical_cols = ['region', 'customer_tier', 'category', 'price_segment']
for col in categorical_cols:
    print(f"\n{col}:")
    print(transactions[col].value_counts().head())
## 2. Sales Performance Analysis"
print("💰 SALES PERFORMANCE ANALYSIS")
print("=" * 50)

# Revenue distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes[0,0].hist(transactions['revenue'], bins=50, alpha=0.7, color='skyblue')
axes[0,0].set_title('Distribution of Transaction Revenue')
axes[0,0].set_xlabel('Revenue ($)')
axes[0,0].set_ylabel('Frequency')

# Revenue by category
category_revenue = transactions.groupby('category')['revenue'].sum().sort_values(ascending=False)
axes[0,1].bar(category_revenue.index, category_revenue.values, color='lightcoral')
axes[0,1].set_title('Total Revenue by Category')
axes[0,1].set_xlabel('Category')
axes[0,1].set_ylabel('Total Revenue ($)')
plt.setp(axes[0,1].xaxis.get_majorticklabels(), rotation=45)

# Revenue by region
region_revenue = transactions.groupby('region')['revenue'].sum()
axes[1,0].pie(region_revenue.values, labels=region_revenue.index, autopct='%1.1f%%')
axes[1,0].set_title('Revenue Distribution by Region')

# Revenue by customer tier
tier_revenue = transactions.groupby('customer_tier')['revenue'].sum()
axes[1,1].bar(tier_revenue.index, tier_revenue.values, color='lightgreen')
axes[1,1].set_title('Revenue by Customer Tier')
axes[1,1].set_xlabel('Customer Tier')
axes[1,1].set_ylabel('Total Revenue ($)')
plt.tight_layout()
plt.show()
# Profitability analysis\n",
print("\n📊 Profitability Analysis:\n")

# Profit by category
profit_by_category = transactions.groupby('category').agg({
    'revenue': 'sum',
    'profit': 'sum',
    'quantity': 'sum'
    }).sort_values('profit', ascending=False)
    
profit_by_category['profit_margin'] = (profit_by_category['profit'] / profit_by_category['revenue']) * 100
print("Top 5 Most Profitable Categories:")
display(profit_by_category.head())
print("\n")
# Visualize profitability
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Profit by category
ax1.bar(profit_by_category.index[:5], profit_by_category['profit'][:5], color='gold')
ax1.set_title('Top 5 Most Profitable Categories')
ax1.set_ylabel('Total Profit ($)')
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

# Profit margin by category
ax2.bar(profit_by_category.index[:5], profit_by_category['profit_margin'][:5], color='lightseagreen')
ax2.set_title('Profit Margin by Category (Top 5)')
ax2.set_ylabel('Profit Margin (%)')
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.show()
## 3. Customer Behavior Analysis"
print("👥 CUSTOMER BEHAVIOR ANALYSIS")
print("=" * 50)

# Customer segmentation analysis
customer_metrics = transactions.groupby('customer_id').agg({
    'revenue': 'sum',
    'transaction_id': 'count',
    'quantity': 'sum',
    'date': ['min', 'max']
    }).round(2)
    
customer_metrics.columns = ['total_spent', 'transaction_count', 'total_items', 'first_purchase', 'last_purchase']
customer_metrics = customer_metrics.reset_index()

# Calculate additional metrics
customer_metrics['avg_order_value'] = customer_metrics['total_spent'] / customer_metrics['transaction_count']
customer_metrics['purchase_frequency'] = customer_metrics['transaction_count'] / 12  # Assuming 12 month period

print("📈 Customer Behavior Summary:")
print(f"• Average Customer Lifetime Value: ${customer_metrics['total_spent'].mean():.2f}")
print(f"• Average Number of Transactions per Customer: {customer_metrics['transaction_count'].mean():.1f}")
print(f"• Average Order Value: ${customer_metrics['avg_order_value'].mean():.2f}")
print("\n")
# Visualize customer behavior
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
# Customer spending distribution
axes[0,0].hist(customer_metrics['total_spent'], bins=50, alpha=0.7, color='purple')
axes[0,0].set_title('Customer Lifetime Value Distribution')
axes[0,0].set_xlabel('Total Spending ($)')
axes[0,0].set_ylabel('Number of Customers')
# Transaction frequency distribution
axes[0,1].hist(customer_metrics['transaction_count'], bins=30, alpha=0.7, color='orange')
axes[0,1].set_title('Transaction Frequency per Customer')
axes[0,1].set_xlabel('Number of Transactions')
axes[0,1].set_ylabel('Number of Customers')
# Average order value distribution
axes[1,0].hist(customer_metrics['avg_order_value'], bins=50, alpha=0.7, color='teal')
axes[1,0].set_title('Average Order Value Distribution')
axes[1,0].set_xlabel('Average Order Value ($)')
axes[1,0].set_ylabel('Number of Customers')
# Customer tier analysis
tier_behavior = transactions.groupby('customer_tier').agg({
    'revenue': 'mean',
    'quantity': 'mean',
    'transaction_id': 'count'
    }).round(2)

tier_behavior.columns = ['avg_revenue_per_tx', 'avg_quantity_per_tx', 'total_transactions']
axes[1,1].bar(tier_behavior.index, tier_behavior['avg_revenue_per_tx'], color=['silver', 'gold', 'brown', 'navy'])
axes[1,1].set_title('Average Revenue per Transaction by Customer Tier')
axes[1,1].set_xlabel('Customer Tier')
axes[1,1].set_ylabel('Average Revenue per Transaction ($)')
plt.tight_layout()
plt.show()
## 4. Product Performance Analysis"
print("📦 PRODUCT PERFORMANCE ANALYSIS")
print("=" * 50)

# Product performance metrics
product_analysis = transactions.groupby(['product_id', 'product_name', 'category', 'price_segment']).agg({
    'quantity': 'sum',
    'revenue': 'sum',
    'profit': 'sum',
    'transaction_id': 'count'
    }).round(2)
    
product_analysis.columns = ['total_quantity', 'total_revenue', 'total_profit', 'transaction_count']
product_analysis = product_analysis.reset_index()
product_analysis['profit_margin'] = (product_analysis['total_profit'] / product_analysis['total_revenue']) * 100

print("🏆 Top 10 Best-Selling Products by Revenue:")
top_products = product_analysis.nlargest(10, 'total_revenue')[['product_name', 'category', 'total_revenue', 'total_quantity', 'profit_margin']]
display(top_products)

print("\n💎 Top 10 Most Profitable Products:")
top_profitable = product_analysis.nlargest(10, 'total_profit')[['product_name', 'category', 'total_profit', 'profit_margin']]
display(top_profitable)
# Product performance visualization\n",
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Revenue by price segment
segment_revenue = product_analysis.groupby('price_segment')['total_revenue'].sum()
axes[0,0].bar(segment_revenue.index, segment_revenue.values, color=['lightblue', 'lightgreen', 'gold', 'lightcoral'])
axes[0,0].set_title('Total Revenue by Price Segment')
axes[0,0].set_ylabel('Total Revenue ($)')

# Quantity sold by category
category_quantity = transactions.groupby('category')['quantity'].sum().sort_values(ascending=False)
axes[0,1].bar(category_quantity.index, category_quantity.values, color='skyblue')
axes[0,1].set_title('Total Quantity Sold by Category')
axes[0,1].set_ylabel('Total Quantity Sold')
plt.setp(axes[0,1].xaxis.get_majorticklabels(), rotation=45)

# Profit margin distribution by category
category_margin = product_analysis.groupby('category')['profit_margin'].mean().sort_values(ascending=False)
axes[1,0].bar(category_margin.index, category_margin.values, color='lightgreen')
axes[1,0].set_title('Average Profit Margin by Category')
axes[1,0].set_ylabel('Average Profit Margin (%)')
plt.setp(axes[1,0].xaxis.get_majorticklabels(), rotation=45)

# Price vs Quantity scatter plot
axes[1,1].scatter(product_analysis['total_revenue'], product_analysis['total_quantity'], 
                 alpha=0.6, c=product_analysis['profit_margin'], cmap='viridis')
axes[1,1].set_xlabel('Total Revenue ($)')
axes[1,1].set_ylabel('Total Quantity Sold')
axes[1,1].set_title('Revenue vs Quantity Sold (Color: Profit Margin)')
plt.colorbar(axes[1,1].collections[0], ax=axes[1,1], label='Profit Margin (%)')
plt.tight_layout()
plt.show()
## 5. Time Series Analysis"
print("📅 TIME SERIES ANALYSIS")
print("=" * 50)

# Monthly trends
monthly_trends = transactions.groupby('month').agg({
    'revenue': 'sum',
    'profit': 'sum',
    'transaction_id': 'count',
    'customer_id': 'nunique'
    }).round(2)
    
monthly_trends.columns = ['monthly_revenue', 'monthly_profit', 'transaction_count', 'unique_customers']
monthly_trends['avg_transaction_value'] = monthly_trends['monthly_revenue'] / monthly_trends['transaction_count']
    
print("📈 Monthly Performance Trends:")
display(monthly_trends.head())
    
print("\nTime series visualization")
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Revenue trend
axes[0,0].plot(monthly_trends.index.astype(str), monthly_trends['monthly_revenue'], 
              marker='o', linewidth=2, color='green')
axes[0,0].set_title('Monthly Revenue Trend')
axes[0,0].set_ylabel('Revenue ($)')
axes[0,0].tick_params(axis='x', rotation=45)

# Profit trend
axes[0,1].plot(monthly_trends.index.astype(str), monthly_trends['monthly_profit'], 
              marker='s', linewidth=2, color='blue')
axes[0,1].set_title('Monthly Profit Trend')
axes[0,1].set_ylabel('Profit ($)')
axes[0,1].tick_params(axis='x', rotation=45)

# Customer count trend
axes[1,0].plot(monthly_trends.index.astype(str), monthly_trends['unique_customers'], 
              marker='^', linewidth=2, color='red')
axes[1,0].set_title('Monthly Unique Customers')
axes[1,0].set_ylabel('Number of Customers')
axes[1,0].tick_params(axis='x', rotation=45)

# Average transaction value trend
axes[1,1].plot(monthly_trends.index.astype(str), monthly_trends['avg_transaction_value'], 
              marker='d', linewidth=2, color='purple')
axes[1,1].set_title('Average Transaction Value Trend')
axes[1,1].set_ylabel('Average Value ($)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()
# Seasonal patterns\n",
print("🌞 Seasonal Analysis:")

# Day of week analysis
transactions['day_of_week'] = transactions['date'].dt.day_name()
day_analysis = transactions.groupby('day_of_week').agg({
    'revenue': 'sum',
    'transaction_id': 'count',
    'customer_id': 'nunique'
    }).round(2)
# Reorder days
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_analysis = day_analysis.reindex(day_order)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
# Revenue by day of week
ax1.bar(day_analysis.index, day_analysis['revenue'], color='lightblue')
ax1.set_title('Revenue by Day of Week')
ax1.set_ylabel('Total Revenue ($)')
ax1.tick_params(axis='x', rotation=45)
# Transactions by day of week
ax2.bar(day_analysis.index, day_analysis['transaction_id'], color='lightcoral')
ax2.set_title('Number of Transactions by Day of Week')
ax2.set_ylabel('Number of Transactions')
ax2.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

print("\n📊 Daily Performance Summary:")
display(day_analysis)
## 6. Geographic Analysis"
print("🌎 GEOGRAPHIC ANALYSIS")
print("=" * 50)

# Regional performance
regional_analysis = transactions.groupby('region').agg({
    'revenue': 'sum',
    'profit': 'sum',
    'transaction_id': 'count',
    'customer_id': 'nunique',
    'quantity': 'sum'
    }).round(2)

regional_analysis.columns = ['total_revenue', 'total_profit', 'transaction_count', 'unique_customers', 'total_quantity']
regional_analysis['avg_transaction_value'] = regional_analysis['total_revenue'] / regional_analysis['transaction_count']
regional_analysis['revenue_per_customer'] = regional_analysis['total_revenue'] / regional_analysis['unique_customers']
print("🏞️ Regional Performance:")
display(regional_analysis)

# Geographic visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Revenue by region
axes[0,0].bar(regional_analysis.index, regional_analysis['total_revenue'], color=['red', 'blue', 'green', 'orange'])
axes[0,0].set_title('Total Revenue by Region')
axes[0,0].set_ylabel('Revenue ($)')

# Customers by region
axes[0,1].bar(regional_analysis.index, regional_analysis['unique_customers'], color=['red', 'blue', 'green', 'orange'])
axes[0,1].set_title('Unique Customers by Region')
axes[0,1].set_ylabel('Number of Customers')
plt.setp(axes[0,1].xaxis.get_majorticklabels(), rotation=45)

# Average transaction value by region
axes[1,0].bar(regional_analysis.index, regional_analysis['avg_transaction_value'], color=['red', 'blue', 'green', 'orange'])
axes[1,0].set_title('Average Transaction Value by Region')
axes[1,0].set_ylabel('Average Value ($)')

# Revenue per customer by region
axes[1,1].bar(regional_analysis.index, regional_analysis['revenue_per_customer'], color=['red', 'blue', 'green', 'orange'])
axes[1,1].set_title('Revenue per Customer by Region')
axes[1,1].set_ylabel('Revenue per Customer ($)')

plt.tight_layout()
plt.show()
## 7. Customer Segmentation Analysis"
print("🎯 CUSTOMER SEGMENTATION ANALYSIS")
print("=" * 50)

# RFM Analysis (Recency, Frequency, Monetary)
latest_date = transactions['date'].max()
rfm_data = transactions.groupby('customer_id').agg({
    'date': lambda x: (latest_date - x.max()).days,  # Recency
    'transaction_id': 'count',                      # Frequency
    'revenue': 'sum'                               # Monetary
    })
    
rfm_data.columns = ['recency', 'frequency', 'monetary']
rfm_data = rfm_data.reset_index()

# Create RFM segments
rfm_data['r_score'] = pd.qcut(rfm_data['recency'], 4, labels=[4, 3, 2, 1])  # Lower recency is better
rfm_data['f_score'] = pd.qcut(rfm_data['frequency'], 4, labels=[1, 2, 3, 4])  # Higher frequency is better
rfm_data['m_score'] = pd.qcut(rfm_data['monetary'], 4, labels=[1, 2, 3, 4])  # Higher monetary is better

rfm_data['rfm_score'] = rfm_data['r_score'].astype(str) + rfm_data['f_score'].astype(str) + rfm_data['m_score'].astype(str)

# Define customer segments
def segment_customer(row):
    if row['rfm_score'] in ['444', '443', '434', '433']:
        return 'Champions'
    elif row['rfm_score'] in ['344', '343', '334', '333', '342', '332']:
        return 'Loyal Customers'
    elif row['rfm_score'] in ['422', '421', '412', '411']:
        return 'New Customers'
    elif row['rfm_score'] in ['244', '243', '234', '233', '242', '232']:
        return 'Potential Loyalists'
    elif row['rfm_score'] in ['144', '143', '134', '133', '142', '132']:
        return 'At Risk'
    elif row['rfm_score'] in ['111', '112', '113', '114']:
        return 'Lost Customers'
    else:
        return 'Regular Customers'

rfm_data['segment'] = rfm_data.apply(segment_customer, axis=1)

print("🎯 Customer Segments Distribution:")
segment_distribution = rfm_data['segment'].value_counts()
print(segment_distribution)

# Visualize customer segments
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Segment distribution
ax1.pie(segment_distribution.values, labels=segment_distribution.index, autopct='%1.1f%%', startangle=90)
ax1.set_title('Customer Segments Distribution')

# Segment value analysis
segment_value = rfm_data.groupby('segment')['monetary'].mean().sort_values(ascending=False)
ax2.bar(segment_value.index, segment_value.values, color='lightgreen')
ax2.set_title('Average Customer Value by Segment')
ax2.set_ylabel('Average Spending ($)')
ax2.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

print("\n💰 Customer Segments Summary:")
segment_summary = rfm_data.groupby('segment').agg({
    'monetary': ['count', 'mean', 'sum'],
    'frequency': 'mean',
    'recency': 'mean'
    }).round(2)
    
segment_summary.columns = ['customer_count', 'avg_spending', 'total_revenue', 'avg_frequency', 'avg_recency']
display(segment_summary)
## 8. Business Insights & Recommendations"
print("💡 BUSINESS INSIGHTS & RECOMMENDATIONS")
print("=" * 60)

# Calculate key business metrics
total_revenue = transactions['revenue'].sum()
total_customers = transactions['customer_id'].nunique()
avg_customer_lifetime = customer_metrics['total_spent'].mean()
top_category = transactions.groupby('category')['revenue'].sum().idxmax()
most_profitable_category = profit_by_category['profit'].idxmax()

print("\n📈 KEY BUSINESS INSIGHTS:")
print("=" * 40)
print(f"1. 💰 Total Business Performance:")
print(f"   • Total Revenue: ${total_revenue:,.2f}")
print(f"   • Total Profit: ${transactions['profit'].sum():,.2f}")
print(f"   • Customer Base: {total_customers:,} unique customers")
print(f"   • Average Customer Lifetime Value: ${avg_customer_lifetime:.2f}")
print("\n")
print(f"\n2. 🏆 Top Performing Areas:")
print(f"   • Highest Revenue Category: {top_category}")
print(f"   • Most Profitable Category: {most_profitable_category}")
print(f"   • Best Performing Region: {regional_analysis['total_revenue'].idxmax()}")
print(f"   • Most Valuable Customer Segment: {segment_value.idxmax()}")
print("\n")
print(f"\n3. 📊 Customer Insights:")
print(f"   • {segment_distribution.idxmax()} segment has the most customers ({segment_distribution.max()} customers)")
print(f"   • {segment_value.idxmax()} segment has highest average spending (${segment_value.max():.2f})")
print(f"   • Customer retention rate: {(len(rfm_data[rfm_data['segment'].isin(['Champions', 'Loyal Customers', 'Potential Loyalists'])]) / len(rfm_data) * 100):.1f}%")
print("\n")
print(f"\n4. 📈 Growth Opportunities:")
# Identify underperforming regions
lowest_region = regional_analysis['total_revenue'].idxmin()
print(f"   • {lowest_region} region has the lowest revenue - consider targeted marketing")
    
# Identify categories with high revenue but low profit margin
low_margin_categories = profit_by_category[profit_by_category['profit_margin'] < profit_by_category['profit_margin'].median()]
if not low_margin_categories.empty:
    print(f"   • {low_margin_categories.index[0]} category has high revenue but low profit margin - review pricing strategy")
    
print("\n🎯 STRATEGIC RECOMMENDATIONS:")
print("=" * 40)
print("1. 🎯 Customer Retention:")
print("   • Focus on 'Loyal Customers' and 'Potential Loyalists' segments")
print("   • Implement loyalty programs for high-value customers")
print("   • Create personalized marketing campaigns for each segment")
print("\n2. 📈 Revenue Optimization:")
print("   • Increase marketing in underperforming regions")
print("   • Optimize pricing for low-margin high-revenue categories")
print("   • Expand product offerings in top-performing categories")
print("\n3. 🚀 Growth Strategies:")
print("   • Target 'New Customers' with onboarding campaigns")
print("   • Reactivate 'At Risk' customers with special offers")
print("   • Analyze seasonal trends to optimize inventory and marketing")
print("\n4. 💰 Profitability Focus:")
print("   • Promote high-margin products more aggressively")
print("   • Review cost structure for low-margin categories")
print("   • Optimize shipping and operations costs by region")
print("\n" + "=" * 60)
print("🎉 EDA COMPLETED SUCCESSFULLY!")
print("\nNext Steps:")
print("• Run 03_feature_engineering.ipynb for advanced feature creation")
print("• Execute SQL queries from business_queries.sql for database analysis")
print("• Build the Streamlit dashboard for interactive visualization")